## Trajectoryless Analysis Workshop Activity

In this notebook, we're running a sample simulation of 
a protein ([1HEL](https://www.rcsb.org/structure/1hel)) in water and analyzing it with live simulation streaming implemented in GROMACS and MDAnalysis.

We'll work up from the most basic IMDClient usage to a complex, live visualization of the simulation,
ending with a challenge to implement your own trajectoryless analysis code.

### Starting the simulation

To run the simulation, first run the following commands in your terminal:
```bash
cd activity
# Preprocess the topology and run the simulation
./run.sh
```

Then, when you see the following line printed to the terminal, 
you can run your analysis code from this notebook:

`IMD: Will wait until I have a connection and IMD_GO orders.`

### Stopping the simulation

For debugging purposes, you can disconnect from the simulation so that it waits for another connection by using the "Interrupt" button in the JupyterLab toolbar above.

You can also run CTRL+C in the terminal to stop the simulation entirely.




### Creating an MDAnalysis Universe and running an analysis loop

The core object in MDAnalysis is a [Universe](https://userguide.mdanalysis.org/stable/universe.html),
which ties together the topology and trajectory. Using IMDClient, the trajectory argument is replaced
with a connection URL to the running simulation.

Here, we create a Universe from the simulation topology file, connect to the running simulation,
and print some data from the simulation.

In [2]:
from imdclient.IMD import IMDReader
import MDAnalysis as mda

u = mda.Universe("sample_simulation/imdgroup.gro", "imd://localhost:8889")

# Get an atom from the topology
# Universe -> AtomGroup -> Atom
atom = u.atoms[0]
print("    time [         position         ] [         velocity         ] [           force          ] [            box           ]")

# Analysis loop
for ts in u.trajectory:
    print(f'{ts.time:8.3f} {atom.position} {atom.velocity} {atom.force} {u.dimensions[0:3]}')

ConnectionError: IMDClient: No handshake packet received

### Performing a simple distance calculation

Next, we'll calculate the distance between two atoms in the protein:
the alpha carbons of the n-terminus (residue 1) and the c-terminus (residue 129).

To learn more about MDAnalysis selections, see https://userguide.mdanalysis.org/stable/selections.html.

In [ ]:
from imdclient.IMD import IMDReader
import MDAnalysis as mda
import numpy as np

u = mda.Universe("sample_simulation/imdgroup.gro", "imd://localhost:8889")

# Universe -> AtomGroup -> Atom
nter = u.select_atoms("resid 1 and name CA").atoms[0]
cter = u.select_atoms("resid 129 and name CA").atoms[0]

for ts in u.trajectory:
    distance = np.sqrt(np.sum((nter.position - cter.position) ** 2))
    print(f'At time {ts.time:4.3f} fs, the n-terminus and c-terminus are {distance:4.3f} angrstroms apart', end='\r')

/home/law/anaconda3/envs/imdclient-test/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Live visualization of an analysis



In [ ]:
from imdclient.IMD import IMDReader
import numpy as np
import MDAnalysis as mda
from graph_utils import LiveTimeseriesGraph

u = mda.Universe("sample_simulation/imdgroup.gro", "imd://localhost:8889")

nter = u.select_atoms("resid 1 and name CA").atoms[0]
cter = u.select_atoms("resid 129 and name CA").atoms[0]

graph = LiveTimeseriesGraph(
    time_window=1.0, # ps
    dt=0.010, # ps
    title='Distance vs. Time',
    y_label='Distance (Å)',
    legend_label='Nter-Cter',
)
    
for ts in u.trajectory:
    distance = np.sqrt(np.sum((nter.position - cter.position) ** 2))
    graph.update(ts.time, distance)




/home/law/anaconda3/envs/imdclient-test/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: 